In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)


In [3]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [6]:
embeddings=OllamaEmbeddings()
db=FAISS.from_documents(docs,embeddings)
db

In [7]:
## quering
query="What does the speaker belive is the main reason the United States should enter the war?"
docs=db.similarity_search(query)
docs[0].page_content

'To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'

In [11]:
##retriever
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'

In [12]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='4d102394-0528-450e-9867-d89a6e15b20c', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
  np.float32(15331.18)),
 (Document(id='f6d5b191-7a53-45f4-8645-5a3f4b727b37', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we s

In [13]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[2.418344259262085,
 -2.102947950363159,
 1.7210966348648071,
 1.1833943128585815,
 -1.3039555549621582,
 0.5048840641975403,
 0.6780940890312195,
 0.4022868871688843,
 1.0672235488891602,
 -2.4043314456939697,
 2.79386568069458,
 0.05201878398656845,
 -1.0869536399841309,
 2.0314252376556396,
 -0.057430900633335114,
 -1.723926067352295,
 -0.6350938081741333,
 1.7429773807525635,
 1.4201807975769043,
 -2.842724561691284,
 -1.098328948020935,
 -1.323011875152588,
 2.0858964920043945,
 -1.3520818948745728,
 1.2085723876953125,
 -1.6475049257278442,
 1.1782598495483398,
 -1.5852110385894775,
 -1.3094418048858643,
 -1.190725326538086,
 1.8230429887771606,
 -2.543802261352539,
 -1.752306342124939,
 6.005941390991211,
 2.100999116897583,
 -4.103967666625977,
 -1.5163410902023315,
 1.7906795740127563,
 -0.676429033279419,
 -0.808976411819458,
 -0.5000529885292053,
 -3.1252856254577637,
 0.47582897543907166,
 0.256515771150589,
 -0.7464157342910767,
 -1.4779093265533447,
 -0.4903891086578369,


In [14]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='4d102394-0528-450e-9867-d89a6e15b20c', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
 Document(id='f6d5b191-7a53-45f4-8645-5a3f4b727b37', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things w

In [16]:
## saving And Loading
db.save_local("faiss_index")

In [17]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [18]:
docs

[Document(id='4d102394-0528-450e-9867-d89a6e15b20c', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
 Document(id='f6d5b191-7a53-45f4-8645-5a3f4b727b37', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things w